In [5]:
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import FixedEmbeddingComposite, EmbeddingComposite
from dwave_networkx.algorithms.max_cut import maximum_cut
from minorminer import find_embedding
import dwave.inspector
import matplotlib.pyplot as plt
import matplotlib as mpl
import networkx as nx
import numpy as np
import os, re
import json
import ast

In [19]:
def read_dict_from_file(fname):
    """
    Parameters
    ----------
    fname : string
        Name of a file that contains a dict
    
    Returns
    -------
    d : dict
        Dictionary contained in the file
    """
    with open(fname, mode='r') as f:
        data = f.read()
    d = ast.literal_eval(data)
    return d

def is_complete(graph):
    """ 
    Parameters
    ----------
    graph : dict
        Form: {edge: weight, ...}
    
    Returns
    -------
    is_complete : bool
        1 --> graph is complete, 0 --> graph is not complete
    """
    num_nodes, num_edges = len(set( np.array(list( graph.keys() )).flatten() )), len(graph.keys())
    return num_edges == num_nodes * (num_nodes - 1) / 2

def normalize(edges):
    """ 
    Parameters
    ----------
    edges : dict
        Form: {edge: weight, ...}
    
    Returns
    -------
    normalized_edges : dict
        edges with normalized edge weights
    """
    max_weight = np.max(np.absolute(list(edges.values())))
    for edge in edges.keys():
        edges[edge] /= max_weight
    return edges

def save_results(response, outfolder):
    """ 
    Saves output of the annealing into 3 files ('embedding.txt', 'solution.txt', 'timing.txt') in outfolder.

    Parameters
    ----------
    response : D-Wave SampleSet
        Outpu of the annealing
    
    Returns
    -------
    None
    """
    
    def mkdir(dirname):
        try:
            os.mkdir(dirname)
        except FileExistsError:
            pass

    mkdir(outfolder)
    
    with open(os.path.join(outfolder, 'embedding.txt'), mode='w') as outfile:
        outfile.write(json.dumps(response.info['embedding_context']['embedding']))
    
    with open(os.path.join(outfolder, 'solution.txt'), mode='w') as outfile:
        d = {'solution': response.first.sample, 'energy': response.first.energy, 'chain_break_fraction': response.first.chain_break_fraction.round(5)}
        outfile.write(str(d))
    
    with open(os.path.join(outfolder, 'timing.txt'), mode='w') as outfile:
        outfile.write(json.dumps(response.info['timing']))

### Connect to a sampler

In [ ]:
num_reads = 100
annealing_time = 30

for i, fname in enumerate(os.listdir(os.path.join(os.getcwd(), 'graphs_python_dictionary'))):

    if i == 1: break
    fname = 'be150.8.1_py.txt'

    print(str(i) + ': ' + fname, end='\t')

    # read problem from a file
    J_dict = read_dict_from_file(os.path.join('graphs_python_dictionary', fname))
    max_chain_strength = np.max(np.absolute(list(J_dict.values())))

    # set up field dict to 0 (DWave sampler wants it)
    h_dict = {}
    for node1, node2 in J_dict.keys():
        h_dict[node2] = 0.0
        h_dict[node1] = 0.0

    # create a problem graph to find embedding
    G = nx.Graph(J_dict.keys())

    # connect to solver
    sampler = DWaveSampler(profile='viderzen_protonmail')
    max_slope = 1 / sampler.properties['annealing_time_range'][0]

    # find embedding of a problem on DWave QPU
    embedding = find_embedding(G, sampler.properties['couplers'], verbose=3)
    
    # sample problem solutions
    sampler = FixedEmbeddingComposite(sampler, embedding)
    response1 = sampler.sample_ising(h_dict, J_dict, num_reads=num_reads, annealing_time=annealing_time, chain_strength=2.5*max_chain_strength)

    # use best solution (min energy) and check for better solutions in its vicinity with reverse annealing
    s = 0.45
    init_state = response1.first.sample
    anneal_schedule = [[0, 1], [(1-s)/max_slope, s], [(1-s)/max_slope + annealing_time, s], [2*(1-s)/max_slope + annealing_time, 1]]
    response2 = sampler.sample_ising(h_dict, J_dict, num_reads=num_reads, chain_strength=2.5*max_chain_strength, anneal_schedule=anneal_schedule, 
                                    initial_state=init_state, reinitialize_state=True)

    # save results and info about results to files
    save_results(response2, os.path.join('results', fname[:-7]))

    print('Done.')